Welcome to the Introduction to LangGraph notebook!

In this notebook, we'll explore how to use the [LangGraph](https://github.com/langchain-ai/langgraph) Python package to build composable, stateful graphs for LLM workflows. We'll walk through step-by-step examples using tools from LangGraph, LangChain, and Gradio to build an interactive experience. You'll see how graphs can help structure conversational or tool-using flows in AI applications.

## What We'll Do

- Install and configure the key packages: `langgraph`, `langsmith`, `langchain` (with OpenAI), `langchain-community`, and `pyppeteer`
- Set up your environment for running LLM graphs, including configuring credentials for LangSmith and OpenAI
- Build and experiment with simple to advanced graph-based workflows

## Setup: Install Dependencies

Before you begin, make sure you have the required packages installed. You can install them with:

```bash
uv add langgraph langsmith "langchain[openai]" langchain-community pyppeteer
```

> **Note**: If you haven't already, [sign up for LangSmith](https://smith.langchain.com/) and create a project. You'll also need to update your environment variables with your API keys to run the examples in this notebook.

Let's get started!

In [ ]:
from pydantic import BaseModel
from dotenv import load_dotenv
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing import Annotated
import random
from IPython.display import Image, display
import gradio as gr
from langchain_openai import ChatOpenAI
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from langgraph.prebuilt import ToolNode, tools_condition
load_dotenv()

In [ ]:
# Some useful constants

nouns = ["Cabbages", "Unicorns", "Toasters", "Penguins", "Bananas", "Zombies", "Rainbows", "Eels", "Pickles", "Muffins"]
adjectives = ["outrageous", "smelly", "pedantic", "existential", "moody", "sparkly", "untrustworthy", "sarcastic", "squishy", "haunted"]

In [ ]:
# Define graph state
# State is the input and output data of the graph
# It is a Pydantic model that defines the structure of the state
class State(BaseModel):
    messages: Annotated[list, add_messages]

In [ ]:
# Define graph builder
# The graph builder is used to build the graph nodes and edges

graph_builder = StateGraph(State)

In [ ]:
# Add first node. It will be a non-llm node that generates a random response

def first_node(state: State) -> State:
    reply = f"{random.choice(nouns)} are {random.choice(adjectives)}"
    messages = [{'role': 'assistant', 'content': reply}]
    return State(messages=messages)

In [ ]:
# Add nodes and edges and compile the graph

graph_builder.add_node("first_node", first_node)
graph_builder.add_edge(START, "first_node")
graph_builder.add_edge("first_node", END)
graph = graph_builder.compile()

In [ ]:
# Display the graph
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Invoke the graph
state = State(messages=[])
graph.invoke(state)

In [ ]:
# Add a chatbot node that uses an llm to generate a response

llm = ChatOpenAI(model="gpt-4o-mini")

def chatbot_node(state: State) -> State:
    response = llm.invoke(state.messages)
    new_state = State(messages=[response])
    return new_state

In [ ]:
# Add nodes and edges and compile the graph
# Display the graph

graph_builder.add_node("chatbot_node", chatbot_node)
graph_builder.add_edge(START, "chatbot_node")
graph_builder.add_edge("chatbot_node", END)
graph = graph_builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Invoke the graph with a query

messages = [{'role': 'user', 'content': 'LangGraph vs CrewAI Flows?'}]
state = State(messages=messages)
graph.invoke(state)

In [ ]:
# Configure serper to be used as a tool

serper = GoogleSerperAPIWrapper()
serper.run("LangGraph vs CrewAI Flows?")

search_tool = Tool(
    name="search_tool",
    func=serper.run,
    description="Use this tool to search the web for information"
)

In [ ]:
# Add tools to LLM
# Adding tools to the LLM allows it to use the tools in the graph

tools = [search_tool]

llm = ChatOpenAI(model="gpt-4o-mini")

llm_with_tools = llm.bind_tools(tools)

In [ ]:
# Define a tool node to execute the tools
# Binding tools with LLM is different from the tool node.
# Binding tools to LLM tells the LLMs about the available tools.
# We need to add a tool node to execute the tools.

tool_node = ToolNode(tools=tools)

In [ ]:
# Rewrite the chatbot node to use the llm with tools

def chatbot_node(state: State) -> State:
    response = llm_with_tools.invoke(state.messages)
    new_state = State(messages=[response])
    return new_state

In [ ]:
# Add nodes and edges to build graph
# Use add_conditional_edges to add edges based on tools_condition
# Don't forget to add an edge from tools_node back to chatbot_node

graph_builder.add_node("chatbot_node", chatbot_node)
graph_builder.add_node("tools_node", tool_node)

graph_builder.add_edge(START, "chatbot_node")
graph_builder.add_conditional_edges("chatbot_node", tools_condition, {"tools": "tools_node", END: END})
graph_builder.add_edge("tools_node", "chatbot_node")
graph_builder.add_edge("chatbot_node", END)
graph = graph_builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Invoke the graph with a query
messages = [{'role': 'user', 'content': 'LangGraph vs CrewAI Flows in 2025?'}]
state = State(messages=messages)
graph.invoke(state)